In [ ]:
import scanpy as sc
import scanpy.external as sce
import pandas as pd
import numpy as np
import os
import triku as tk
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
import ray
import subprocess

from IPython.display import display, HTML

from tqdm.notebook import tqdm

from bokeh.io import show, output_notebook, reset_output

reset_output()
output_notebook()

In [ ]:
seed = 10

In [ ]:
data_dir = 'data/'
fig_dir = 'figures/'

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
def assign_cats(adata, dict_cats, column_groupby='leiden', quantile_gene_sel=0.7, do_return=False, intermediate_states=False, diff=0.05, 
                key_added='assigned_cats', min_score=0.6, others_name='unassigned'):
    """
    This functions uses a set of genes assigned to different categories so that leiden clusters can be assigned to one of these categories.
    For example, to categorize fibroblasts from pericytes, endothelial cells, or cells with high mitochondrial content.
    It could be done with each cell individually, but it is better to use clusters to discern the different categories because
    the method, although efficient, can sometimes be noisy due to the noisiness of the sc datasets.
    """
    
    for cat in list(dict_cats.keys()):
        mat_cat = np.zeros((len(adata), len(dict_cats[cat])), dtype=float)
        
        for gene_idx, gene in enumerate(dict_cats[cat]):
            try:
                mat_cat[:, gene_idx] = np.asarray(np.dot(adata.obsp['connectivities'], adata[:, gene].X).todense()).ravel() / adata.uns['neighbors']['params']['n_neighbors']
                mat_cat[mat_cat[:, gene_idx] > 0, gene_idx] = np.argsort(np.argsort(mat_cat[mat_cat[:, gene_idx] > 0, gene_idx]))
                mat_cat[:, gene_idx] /= np.max(mat_cat[:, gene_idx])
            except:
                print(f'Gene {gene} is not on the list')    
            
        sum_mat_cat = np.asarray(mat_cat.mean(1)).ravel()       
        adata.obs[cat] = sum_mat_cat
    
    score_per_cluster = adata.obs[[column_groupby] + list(dict_cats.keys())].groupby(column_groupby).quantile(quantile_gene_sel)
    max_cat_dict_std = dict(zip(score_per_cluster.std(1).index, score_per_cluster.std(1).values))
    adata.obs[f'{key_added}_std'] = [max_cat_dict_std[i] for i in adata.obs[column_groupby]]
    max_cat_dict_mean = dict(zip(score_per_cluster.mean(1).index, score_per_cluster.mean(1).values))
    adata.obs[f'{key_added}_mean'] = [max_cat_dict_mean[i] for i in adata.obs[column_groupby]]
    max_cat_dict_max = dict(zip(score_per_cluster.max(1).index, score_per_cluster.max(1).values))
    adata.obs[f'{key_added}_max'] = [max_cat_dict_max[i] for i in adata.obs[column_groupby]]
    adata.obs[f'{key_added}_CV'] = adata.obs[f'{key_added}_mean'] / adata.obs[f'{key_added}_std']
    
    for cat in score_per_cluster.columns:
        max_cat_dict = dict(zip(score_per_cluster.index, score_per_cluster[cat].values))        
        adata.obs[f'{key_added}_{cat}'] = [max_cat_dict[i] for i in adata.obs[column_groupby]]
    
    if intermediate_states: # For each cluster we will identify which categories are close to the highest one, and merge their names.
        list_names_cats_per_cluster = []
        for cluster in score_per_cluster.index:
            scores_cluster = score_per_cluster.loc[cluster]
            scores_cluster = scores_cluster[scores_cluster > scores_cluster.max() - diff]
            list_names_cats_per_cluster.append('/'.join(scores_cluster.index))
        
        final_cat_dict = dict(zip(score_per_cluster.idxmax(axis=1).index, list_names_cats_per_cluster))
    else:        
        final_cat_dict = dict(zip(score_per_cluster.idxmax(axis=1).index, score_per_cluster.idxmax(axis=1).values))
    
    adata.obs[f'{key_added}'] = [str(final_cat_dict[i]) for i in adata.obs[column_groupby]]
    
    adata.obs[f'{key_added}'][adata.obs[f'{key_added}_max'] < min_score] = others_name
    
    if do_return:
        return score_per_cluster

In [ ]:
de_micheli_mouse = sc.read('data/processed/de_micheli_mouse_d0.h5')
de_micheli_human = sc.read('data/processed/de_micheli_human.h5')

de_micheli_mouse.var_names = [i.upper() for i in de_micheli_mouse.var_names]

de_micheli_mouse.var_names_make_unique()
de_micheli_human.var_names_make_unique()

In [ ]:
de_micheli = sc.AnnData.concatenate(de_micheli_mouse, de_micheli_human, batch_key='organism', 
                                    batch_categories=['mouse', 'human'], join='outer')

In [ ]:
sc.pp.pca(de_micheli, random_state=seed, n_comps=30)
sce.pp.bbknn(de_micheli, neighbors_within_batch=2, batch_key='batch', approx=False, set_op_mix_ratio=0.2)
tk.tl.triku(de_micheli, n_procs=1, random_state=seed, use_adata_knn=True)

In [ ]:
sc.tl.umap(de_micheli, min_dist=0.3, random_state=seed)
sc.tl.leiden(de_micheli, resolution=2, random_state=seed)
sc.pl.umap(de_micheli, color=['leiden', 'batch', 'organism', 'total_counts'], legend_loc='on data', ncols=1)

In [ ]:
A_markers = ['6030408B16Rik', 'Adamtsl2', 'Cdh19', 'Cdkn2b', 'Col18a1', 'Col26a1', 
             'Col9a2', 'Dlk1', 'Fetub', 'Gfra2', 'Gm11681', 'Gpld1', 'Greb1', 'Gria1', 
             'Kcnb2', 'Kcnk2', 'Mpzl2', 'Ngfr', 'Plppr4', 
             'Ptgfr', 'Rgs17', 'Saa1', 'Saa2', 'Shisa3', 'Sipa1l1', 'Sorcs2', 'Sox9', 
             'Sphkap', 'Syndig1', 'Trpm6']
B_markers = ['Cldn1', 'Crabp2', 'Dleu7', 'Efnb3', 'Gjb5', 'Grin2b', 'Itgb4', 'Kcnj13', 
             'Kcnj2', 'Lgals7', 'Lypd2', 'Mansc4', 'Moxd1', 'Mpzl2', 'Perp', 'Prodh', 'Ptch1', 
             'Slc6a13', 'Stra6', 'Tec', 'Tenm2', 'Wnt10a', 'Wnt6']

In [ ]:
sc.pl.umap(de_micheli, color=['leiden', 'organism'], ncols=1, cmap=magma)

In [ ]:
sc.pl.umap(de_micheli, color=[i.upper() for i in A_markers if i.upper() in de_micheli.var_names], ncols=3, cmap=magma)

In [ ]:
sc.pl.umap(de_micheli, color=[i.upper() for i in B_markers if i.upper() in de_micheli.var_names], ncols=3, cmap=magma)

**No mapping is available and the markers in human do not correlate to mouse**